# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:

print(os.getcwd())

filepath =  '/home/workspace/event_data'
for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))


/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = []     
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
from cassandra.cluster import Cluster
cluster = Cluster()
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [19]:
### Query1

Getting the artist, song title and length for session ID = 338 and Item ID = 4

For this query session_id and item_in_session are considered as the composite key, because this table will be queried by session_id and item_in_session, together they provide uniqueness of 
Primary Key.

In [8]:
query = "CREATE TABLE IF NOT EXISTS songs_listened_history"
query = query + "(session_id INT, item_in_session INT, artist_name TEXT, song_title TEXT, length FLOAT, PRIMARY KEY (session_id,item_in_session ))"
try:
    session.execute(query)
except Exception as e:
    print(e)

                    

In [10]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songs_listened_history (session_id, item_in_session, artist_name, song_title, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [11]:
query = "SELECT artist_name, song_title, length  FROM songs_listened_history WHERE session_id = 338 AND item_in_session = 4"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name, row.song_title, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### Output of Query1:
> Artist Name: **Faithless**
> Song Title: **Music Matters (Mark Knight Dub)**
> Length of the song: **495.3** 


### Query2
Getting the artist, song title, first name, last name of user with user ID = 10 and session ID = 182 

For this query user_id and session_id are considered as the composite key, because this table will be queried by user_id and session_id, together they provide uniqueness of Primary Key. Item In Session serves as sorting parameter.

In [12]:
query = "CREATE TABLE IF NOT EXISTS users_listened_history"
query = query + "(user_id INT, session_id INT, item_in_session INT, artist_name TEXT, song_title TEXT, first_name TEXT, last_name TEXT,   PRIMARY KEY ((user_id, session_id), item_in_session ));"
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [13]:
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO users_listened_history (user_id, session_id, item_in_session, artist_name, song_title, first_name, last_name)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [14]:
query = "SELECT artist_name, song_title,first_name, last_name FROM users_listened_history WHERE user_id = 10 AND session_id = 182"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name, row.song_title, row.first_name, row.last_name)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [ ]:

### Output of Query2:

With user ID = 10 and for the session = 182 there are 4 songs listened by this user:
    > **Keep On Keepin**;
    > **Greece 2000**;
    > **Kilometer**;
    > **Catch You Baby (Steve Pitron & Max Sanna Radio Edit)**


In [ ]:

### Query3

Getting the  first name and last name of user with song title = 'All Hands Against His Own'

For this query song_title and user_id are considered as the composite key, because this table will be queried by song name and user ID together with
song title gives uniqueness of Primary Key. 

In [15]:
query = "CREATE TABLE IF NOT EXISTS songs_listened_by_users"
query = query + "(song_title TEXT, user_id INT, first_name TEXT, last_name TEXT,   PRIMARY KEY (song_title, user_id ));"
try:
    session.execute(query)
except Exception as e:
    print(e)

                    

In [16]:
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songs_listened_by_users (song_title, user_id, first_name, last_name)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [17]:
query = "SELECT first_name, last_name FROM songs_listened_by_users WHERE song_title = 'All Hands Against His Own'"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.first_name, row.last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


In [ ]:

### Output of Query3:

There are 3 users in total who listened 'All Hands Against His Own':
> User 1: **Jacqueline Lynch**
> User 2: **Tegan Levine**
> User 3: **Sara Johnson**

### Drop the tables before closing out the sessions

In [11]:
## Drop the table before closing out the sessions
try:
    session.execute("DROP TABLE IF EXISTS songs_listened_by_users")
except Exception as e:
    print(e)
    
try:
    session.execute("DROP TABLE IF EXISTS users_listened_history")
except Exception as e:
    print(e)
    
try:
    session.execute("DROP TABLE IF EXISTS songs_listened_history")
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [12]:
session.shutdown()
cluster.shutdown()